In [14]:
import json
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf


In [15]:
@udf('string')
def munge_event(event_as_json):
    event = json.loads(event_as_json)
    event['Host'] = "BHUVNESH-MACHINE" # silly change to show it works
    event['Cache-Control'] = "no-cache"
    return json.dumps(event)

In [16]:
spark = SparkSession \
        .builder \
        .appName("TransformEventsJob") \
        .getOrCreate()

In [17]:
raw_events = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [18]:
raw_events.show()


+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 48 6F 73 7...|events|        0|     0|2018-08-04 10:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     1|2018-08-04 10:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     2|2018-08-04 10:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     3|2018-08-04 10:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     4|2018-08-04 10:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     5|2018-08-04 10:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     6|2018-08-04 10:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     7|2018-08-04 10:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0

In [19]:
raw_events.printSchema()


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [20]:
munged_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .withColumn('munged', munge_event('raw'))


In [21]:
    munged_events.show()

+--------------------+--------------------+--------------------+
|                 raw|           timestamp|              munged|
+--------------------+--------------------+--------------------+
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|2018-08-04 10:04:...|{"Host": "BHUVNES...|
|{"Host": "user1.c...|201

In [22]:
    munged_events.printSchema()


root
 |-- raw: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- munged: string (nullable = true)



In [23]:
 events = raw_events.select(raw_events.value.cast('string'))
extracted_events = events.rdd.map(lambda x: json.loads(x.value)).toDF()


/spark-2.2.0-bin-hadoop2.6/python/pyspark/sql/session.py:351: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [24]:
    extracted_events.show()


+------+-----------------+---------------+--------------+
|Accept|             Host|     User-Agent|    event_type|
+------+-----------------+---------------+--------------+
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|       default|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|
|   */*|user1.

In [25]:
    extracted_events.printSchema()


root
 |-- Accept: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- User-Agent: string (nullable = true)
 |-- event_type: string (nullable = true)



In [26]:
    extracted_events \
        .write \
        .mode("overwrite") \
        .parquet("/tmp/extracted_events")